In [53]:
import pandas as pd
import numpy as np
import zipfile as zf
import os
import matplotlib.pyplot as plt
from pprint import pprint

In [54]:
path = 'data/Raw'
zip = [file for file in os.listdir(path) if file.split('.')[-1] == 'zip'][0]
zip

'acidentes2023.zip'

In [55]:
with zf.ZipFile(f'{path}/{zip}') as folder:
    file = folder.namelist()[0]
    with folder.open(file) as csv_file:
        df = pd.read_csv(csv_file, delimiter = ';', encoding='latin_1')
df = df.replace(',', '.', regex=True) 
df = df.replace('nan', np.nan, regex=True)
df.fillna(-1, inplace=True)
df.head()      

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,...,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop
0,496519,1082160,2023-01-01,domingo,02:00:00,ES,101.0,114,SOORETAMA,Ausência de reação do condutor,...,Masculino,0,1,0,0,-19.09484877,-40.05095848,SPRF-ES,DEL04-ES,UOP01-DEL04-ES
1,496543,1082181,2023-01-01,domingo,03:40:00,SP,116.0,113.1,TAUBATE,Entrada inopinada do pedestre,...,Não Informado,0,0,0,0,-23.0445658,-45.58259814,SPRF-SP,DEL02-SP,UOP02-DEL02-SP
2,496590,1082406,2023-01-01,domingo,01:40:00,MT,163.0,1112,GUARANTA DO NORTE,Reação tardia ou ineficiente do condutor,...,Masculino,0,0,1,0,-9.70020602,-54.87588757,SPRF-MT,DEL06-MT,UOP03-DEL06-MT
3,496610,1082326,2023-01-01,domingo,10:40:00,PR,376.0,314.8,ORTIGUEIRA,Velocidade Incompatível,...,Masculino,1,0,0,0,-23.985512,-51.083555,SPRF-PR,DEL07-PR,UOP02-DEL07-PR
4,496659,1082547,2023-01-01,domingo,14:55:00,MG,116.0,569.4,MANHUACU,Acumulo de água sobre o pavimento,...,Masculino,1,0,0,0,-20.10007457,-42.17884091,SPRF-MG,DEL06-MG,UOP03-DEL06-MG


In [56]:
bool_cols = ['ilesos','feridos_leves', 'feridos_graves', 'mortos', 'uso_solo']
drop_cols = ['regional', 'delegacia', 'uop']
float_cols = ['latitude', 'longitude', 'km']
int_cols = ['idade', 'ano_fabricacao_veiculo', 'br']
cat_cols = ['dia_semana', 'sexo', 'estado_fisico', 'tipo_envolvido', 
            'tipo_veiculo', 'tipo_pista',
            'condicao_metereologica', 'sentido_via',
            'fase_dia', 'classificacao_acidente',
            'tipo_acidente', 'causa_acidente']

df[bool_cols] = df[bool_cols].astype('bool')
df[cat_cols] = df[cat_cols].astype('category')
df[float_cols] = df[float_cols].astype('float64')
df[int_cols] = df[int_cols].astype('int16')
df.marca = df.marca.astype(str)
#df.data_inversa = pd.to_datetime(df.data_inversa)
df.drop(columns=drop_cols, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166833 entries, 0 to 166832
Data columns (total 32 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   id                      166833 non-null  int64   
 1   pesid                   166833 non-null  int64   
 2   data_inversa            166833 non-null  object  
 3   dia_semana              166833 non-null  category
 4   horario                 166833 non-null  object  
 5   uf                      166833 non-null  object  
 6   br                      166833 non-null  int16   
 7   km                      166833 non-null  float64 
 8   municipio               166833 non-null  object  
 9   causa_acidente          166833 non-null  category
 10  tipo_acidente           166833 non-null  category
 11  classificacao_acidente  166833 non-null  category
 12  fase_dia                166833 non-null  category
 13  sentido_via             166833 non-null  category
 14  cond

In [62]:
df['momento'] = pd.to_datetime(df.horario+' '+df.data_inversa, format = "%H:%M:%S %Y-%m-%d")
df.dia_semana = df.dia_semana.cat.set_categories(['domingo', 'segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira', 'sábado'])
#_.drop(columns=['data', 'horario'], inplace=True)

In [64]:
df.to_parquet(f'{path[:-4]}/acidentes.parquet')